In [3]:
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from time import sleep
from itertools import compress

volkws=['btu','bls','bcf','mcf','cfe']
prodkws=['bbls)', 'bcf)','bcfs)',"cfe)","mcf)","mcfs)"]
drvkws=['swap','swaps','put','puts','collar','collars','Henry Hub']
price=['average','price']

In [ ]:
### the code is still under development; descriptions of each step need to be added

In [4]:
def table_scrapper_v2(article):
    tables=article.find_all('table')
    tb_count=1 #bug prevention
    df_list=[]
    des_list=[]
    index_list=[]
    for ii in range(len(tables)):
        temp1=""
        temp="".join(map(str.lower, tables[ii].get_text())) #(("fixed-price" in temp) or (("fixed price" in temp)))\
        cond1= any(volkw in temp for volkw in volkws) and any(drvkw in temp for drvkw in drvkws)
        cond2=False
        if cond1==False: #go to the previous paragraph
            for kth in range(6):
                if len(tables[ii].find_all_previous()[kth].get_text())>15: break
            temp1="".join(map(str.lower, [s.get_text(strip=True) for s \
                in tables[ii].find_all_previous()[:kth+1]]))
            cond1= any(drvkw in re.split(r'\W+',temp) for drvkw in drvkws) and any(volkw in temp1 for volkw in volkws)
            cond2= any(drvkw in re.split(r'\W+',temp1) for drvkw in drvkws) and any(volkw in temp for volkw in volkws)
        cond3= all(x not in temp for x in ['balance sheet','income','liabiliti','earnings']) and (all(pricekw in temp for pricekw in price) or all(pricekw in temp1 for pricekw in price))
        if (cond1 or cond2):
            rows=tables[ii].find_all("tr")
            l=len(rows)
            if l>2: #skip tables that are actuallly paragraph
                lst=[]
                lst_colspan=[None]*l
                i=0
                for row in rows:
                    lst_temp=[cell.get_text(strip=True) for cell in row.find_all("td")+row.find_all("th")]
                    #             print(len(lst_temp),lst_temp)
                    if lst_temp is None: 
                        continue #bug prevention: empty rows
                    #                 elif len(lst_temp)>25: continue #bug prevention?
                    else:
                        lst_colspan[i]=[cell.get('colspan') for cell in row.find_all("td")+row.find_all("th")]
                        if lst_colspan[i].count(None)!=len(lst_colspan[i]): 
                            act=0
                            for j,x in enumerate(lst_colspan[i]):
                                if x!=None:
                                    for _ in range(int(x)-1):
                                        lst_temp.insert(j+act,'')
                                        act+=1
                        lst.append(lst_temp)
                    #                 print(lst_temp)
                for k in range(len(lst)):
                    for j in range(len(lst[k])):
#                         if re.search('\$(?=.*[0-9])',lst[k][j]): 
#                             lst[k][j]=re.sub(r'[^\d.-–\$]',"",lst[k][j]) #$ followed by a number
                        lst[k][j]=re.sub('–', '-',lst[k][j])
                        lst[k][j]=re.sub('[^\w\-\$\(\)% ]', '',lst[k][j])
                        if re.search('[\w%\$]', lst[k][j])==None:
                            lst[k][j]=''
                df = pd.DataFrame(lst)
                df = df.loc[:,df.apply(lambda x: x.str.contains('\w')).any(axis=0)] #delete columns that have no '\w'
                mask = df.iloc[:,-1].str.contains('[\w%\$]') #shift if the last colum is empty
                if sum(filter(None,mask))<df.shape[0]/2:
                    df.loc[mask,df.columns!=df.columns[-1]]=df.loc[mask,].iloc[:,1:].values
                    df=df.iloc[:,:-1]
                df = df.loc[:,df.apply(lambda x: x.str.contains('\w')).any(axis=0)] #delete columns that have no '\w'
                df.columns = df.iloc[0]
                df=df.drop(df.index[0])
                df_list.append(df)
                des_list.append(temp1)
                index_list.append(cond3)
    return (des_list,df_list,index_list)

In [5]:
def wipe(x):
    if re.search('[\w%\$]', x)==None: 
        return ""
    else:
        return x
def table_scrapper_volume(article):
    tables=article.find_all('table')
    tb_count=1 #bug prevention
    df_list=[]
    des_lst=[]
    index_list=[]
    
    for ii in range(len(tables)):
        temp1=""
        temp="".join(map(str.lower, tables[ii].get_text())) #(("fixed-price" in temp) or (("fixed price" in temp)))\
        cond1= (any(volkw in temp for volkw in prodkws) and any(volkw in temp for volkw in ['production','commmitment'])) 
        cond2=False
        if cond1==False: #go to the previous paragraph
            for kth in range(6):
                if len(tables[ii].find_all_previous()[kth].get_text())>15: break
            temp1="".join(map(str.lower, [s.get_text(strip=True) for s \
                in tables[ii].find_all_previous()[:kth+1]]))
#             print('des type',type(temp1),type(temp))
            cond1= any(volkw in temp for volkw in prodkws) and 'production' in temp1
            cond2= any(volkw in temp1 for volkw in prodkws) and 'production' in temp
        cond3= any(volkw in temp for volkw in  ['swap','collar','reserve','realized price','income'])  # delete redundant table
        if (cond1 or cond2):
            rows=tables[ii].find_all("tr")
#             print(ii)
            l=len(rows)
            if l>2: #skip tables that are actuallly paragraph
                lst=[]
                lst_colspan=[None]*l
                i=0
                for row in rows:
                    lst_temp=[cell.get_text(strip=True) for cell in row.find_all("td")+row.find_all("th")]
                    #             print(len(lst_temp),lst_temp)
                    if lst_temp is None: 
                        continue #bug prevention: empty rows
                    #                 elif len(lst_temp)>25: continue #bug prevention?
                    else:
                        lst_colspan[i]=[cell.get('colspan') for cell in row.find_all("td")+row.find_all("th")]
                        if lst_colspan[i].count(None)!=len(lst_colspan[i]): 
                            act=0
                            for j,x in enumerate(lst_colspan[i]):
                                if x!=None:
                                    for _ in range(int(x)-1):
                                        lst_temp.insert(j+act,'')
                                        act+=1
                        lst.append(lst_temp)
                    #                 print(lst_temp)
                for k in range(len(lst)):
                    for j in range(len(lst[k])):
#                         if re.search('\$(?=.*[0-9])',lst[k][j]): 
#                             lst[k][j]=re.sub(r'[^\d.-–\$]',"",lst[k][j]) #$ followed by a number
                        lst[k][j]=re.sub('–', '-',lst[k][j])
                        lst[k][j]=re.sub('[^\w\-\$\(\)% ]', '',lst[k][j])
                        if re.search('[\w%\$]', lst[k][j])==None:
                            lst[k][j]=''
                df = pd.DataFrame(lst)
#                 df = df.applymap(lambda x: re.sub('–', '-',x))
#                 df = df.applymap(wipe) #wipe no-char to ""
                df = df.loc[:,df.apply(lambda x: x.str.contains('\w')).any(axis=0)] #delete columns that have no '\w'
                mask = df.iloc[:,-1].str.contains('[\w%\$]') #shift if the last colum is empty
                if sum(filter(None,mask))<df.shape[0]/2:
                    df.loc[mask,df.columns!=df.columns[-1]]=df.loc[mask,].iloc[:,1:].values
                    df=df.iloc[:,:-1]
                df = df.loc[:,df.apply(lambda x: x.str.contains('\w')).any(axis=0)] #delete columns that have no '\w'
                df.columns = df.iloc[0]
                df=df.drop(df.index[0])
                df_list.append(df)
                des_lst.append(temp1)
                cond3=not cond3
                index_list.append(cond3)
    return (des_lst,df_list,index_list)

In [6]:
def convert_neg_new(string):
    if not string is np.nan:
        if re.search('^\(\d+', str(string)):
            string = re.sub('[\(\),]',"",str(string))
            string = 'Neg'+string
            return string
        elif re.search('^\d+', str(string)):
            string = re.sub(',',"",str(string))
            return string
        else:
            return string
    else:
        return string

def output_table(article,tikcer,fdate,form, path,url):
    if path=='hedge/':
        des_lst,df_lst,index_lst=table_scrapper_v2(article)
    elif path=='volume/':
        des_lst,df_lst,index_lst=table_scrapper_volume(article)
    if len(index_lst)>1:
        if len(list(compress(df_lst, index_lst)))>0:
            des_lst=list(compress(des_lst, index_lst))
            df_lst=list(compress(df_lst, index_lst))
    if len(index_lst)>=1:
        with open('./'+path+form+'/'+ticker+str(int(fdate))+'.csv','w') as f:
            pass
    for i in range(len(df_lst)):
        des=des_lst[i]
        df = df_lst[i].apply(lambda x: x.str.strip())
        df1=df.applymap(convert_neg_new)
        with open('log_second'+'.txt','a') as f:
            f.writelines('good'+ '\n')
        with open('./'+path+form+'/'+ticker+str(int(fdate))+'.csv','a') as f:
            try: df1.to_csv(f,encoding='cp1252')
            except: df1.to_csv(f,encoding='utf-8')
    if len(df_lst)>1:
        index=[df.shape[0]*df.shape[1] for df in df_lst]
        zipped_pairs = zip(index, df_lst)
        df=sorted(zipped_pairs,reverse=True)[0][1] #choose the biggest table
        zipped_pairs = zip(index, des_lst)
        des=sorted(zipped_pairs,reverse=True)[0][1]
    else:
        df=df_lst[0]
        des=des_lst[0]
    with open('./'+path+'all_second'+'.csv','a') as f:
        f.writelines('\n' + ticker+str(int(fdate)) + '\n')
        f.writelines(url + '\n')
        des=re.sub('[\n]',"",des)
        des=re.sub(' +'," ",des)
        f.writelines('"'+des+'"'+'\n')
        try: df.to_csv(f,encoding='cp1252')
        except: df.to_csv(f,encoding='utf-8')

In [7]:
# used to generate a list of gvkeys for hedgers
file_index=pd.read_csv('./data/sec_filing_final_wlinks.csv') # from Stata
files=file_index.loc[~np.isnan(file_index.fdate)].reset_index()
# files[~files.index.isin(index_lst)].to_csv('2016_list.csv')

In [ ]:
# new version
with open('log_second'+'.txt','w') as f: pass
with open('./'+'volume/'+'all_second'+'.csv','w') as f: pass
with open('./'+'hedge/'+'all_second'+'.csv','w') as f: pass
bad_lst1=np.zeros(len(files))
bad_lst2=np.zeros(len(files))
for jj in range(1144,len(files)):
    for path in ['volume/','hedge/']:
        fname=files.fname[jj]
        ticker=files.ticker[jj]
        fdate=files.fdate[jj]
        comnam=files.comnam[jj]
        form=files.form[jj]
        url=files.html[jj]
        with open('log_second'+'.txt','a') as f:
            f.writelines( str(jj)+ '\n')
    #     response = requests.get('https://www.sec.gov/Archives/'+fname)
        try:
            article_path='./data/sample/'+comnam+str(int(fdate))
            with open(article_path, 'rb') as f:
                article = f.read()
                article = BeautifulSoup(article, 'html.parser')
            output_table(article,ticker,fdate,form,path,url)
        except: #complement w/ html
            try:
                article_path='./data/html_data/'+comnam+str(int(fdate))
                with open(article_path, 'rb') as f:
                    article = f.read()
                    article = BeautifulSoup(article, 'html.parser')
                output_table(article,ticker,fdate,form,path,url)
            except:
                with open('log_second'+'.txt','a') as f:
                    f.writelines('\n' + path+'bad' + '\n')
#                 print(path+'bad')
                if path=="volume/": bad_lst1[jj]=1
                if path=="hedge/": bad_lst2[jj]=1
                sleep(1)

In [ ]:
###################final code ends here###################

In [ ]:
#the rest is for debugging